In [7]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import UpSampling2D, MaxPooling1D, Conv2DTranspose, LSTM, UpSampling1D, AveragePooling1D
from tensorflow.keras.layers import Flatten, Conv1DTranspose, Dropout, Input, Conv1D, LeakyReLU, MaxPool1D, Bidirectional, TimeDistributed, Dense, Reshape

In [4]:
STEPSSIZE=15
INPUT_DIM=2
KERNEL_SIZE=2

In [ ]:
x = Input(shape=(STEPSSIZE, INPUT_DIM))
COV1D_NFILTER =32
############# Encoder ##############
# Conv1D
encoder_layer_conv1d = Conv1D(filters=COV1D_NFILTER, kernel_size=KERNEL_SIZE, padding='same', strides=1, activation='linear')(x)
encoder_conv1d_activationFunction = LeakyReLU()(encoder_layer_conv1d)
#BiLSTM
encoder_layer_bilstm = Bidirectional(LSTM(COV1D_NFILTER, return_sequences=True), merge_mode='sum')(encoder_conv1d_activationFunction)
encoder_bilstm_activationFunction = LeakyReLU()(encoder_layer_bilstm)
#BiLSTM 2
encoder_layer_bilstm_2 = Bidirectional(LSTM(1, return_sequences=True), merge_mode='sum')(encoder_bilstm_activationFunction)
encoder_bilstm_2_activationFunction = LeakyReLU()(encoder_layer_bilstm_2)

############# Decoder ##############
decoder_layer_conv1dtranspose = Conv1DTranspose(filter= COV1D_NFILTER, name='', kernel_size=KERNEL_SIZE, padding='same', strides=5, activation='linear')(encoder_bilstm_2_activationFunction)
decoder_conv1dtranspose_activationFunction = LeakyReLU()(decoder_layer_conv1dtranspose)








In [5]:
input_dim=1
timesteps=100
n_filters=30
kernel_size=10
strides=1
pool_size=10
n_units=[25, 1]


In [8]:
x = Input(shape=(timesteps, input_dim), name='input_seq')

# Encoder
encoded = Conv1D(n_filters, kernel_size, strides=strides, padding='same', activation='linear')(x)
encoded = LeakyReLU()(encoded)
encoded = MaxPooling1D(pool_size)(encoded)
encoded = Bidirectional(LSTM(n_units[0], return_sequences=True), merge_mode='sum')(encoded)
encoded = LeakyReLU()(encoded)
encoded = Bidirectional(LSTM(n_units[1], return_sequences=True), merge_mode='sum')(encoded)
encoded = LeakyReLU(name='latent')(encoded)

# Decoder
decoded = Reshape((-1, 1, n_units[1]), name='reshape')(encoded)
decoded = UpSampling2D((pool_size, 1), name='upsampling')(decoded)  #decoded = UpSampling1D(pool_size, name='upsampling')(decoded)
decoded = Conv2DTranspose(input_dim, (kernel_size, 1), padding='same', name='conv2dtranspose')(decoded)
output = Reshape((-1, input_dim), name='output_seq')(decoded)  #output = Conv1D(1, kernel_size, strides=strides, padding='same', activation='linear', name='output_seq')(decoded)

# AE model
autoencoder = Model(inputs=x, outputs=output, name='AE')
autoencoder.summary()
# Encoder model
encoder = Model(inputs=x, outputs=encoded, name='encoder')
encoder.summary()
# Create input for decoder model
encoded_input = Input(shape=(timesteps // pool_size, n_units[1]), name='decoder_input')

# Internal layers in decoder
decoded = autoencoder.get_layer('reshape')(encoded_input)
decoded = autoencoder.get_layer('upsampling')(decoded)
decoded = autoencoder.get_layer('conv2dtranspose')(decoded)
decoder_output = autoencoder.get_layer('output_seq')(decoded)

# Decoder model
decoder = Model(inputs=encoded_input, outputs=decoder_output, name='decoder')
decoder.summary()

Model: "AE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_seq (InputLayer)      [(None, 100, 1)]          0         
                                                                 
 conv1d_1 (Conv1D)           (None, 100, 30)           330       
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 100, 30)           0         
                                                                 
 max_pooling1d (MaxPooling1D  (None, 10, 30)           0         
 )                                                               
                                                                 
 bidirectional (Bidirectiona  (None, 10, 25)           11200     
 l)                                                              
                                                                 
 leaky_re_lu_2 (LeakyReLU)   (None, 10, 25)            0        